In [1]:
%load_ext autoreload
%autoreload 2

# Parameter Definition

In [2]:
from skyllh.core.parameters import Parameter

Creating a fixed parameter
---

A fixed parameter has a name and an initial value. Its `isfixed` property is set to `True`.

In [3]:
param_a = Parameter(name='a', initial=2.1)
print(param_a)
print(param_a.isfixed)

Parameter: a = 2.100 [fixed]
True


Changing the value of a fixed parameter
---

In [4]:
param_a.change_fixed_value(3.3)
print(param_a)

Parameter: a = 3.300 [fixed]


Creating a floating parameter
---

A floating parameter has a name, an initial value, and a value range from `valmin` to `valmax`. Its `isfixed` property is set to `False`.  

In [5]:
param_b = Parameter(name='b', initial=6.3, valmin=4.0, valmax=7.3)
print(param_b)
print(param_b.isfixed)

Parameter: b = 6.300 [floating] {
    initial: 6.300
    range: (4.000, 7.300)
}
False


Converting fixed and floating parameters
---

In [6]:
# Converting a fixed parameter into a floating parameter.
param_a.make_floating(initial=3.3, valmin=0, valmax=4)
print(param_a)

Parameter: a = 3.300 [floating] {
    initial: 3.300
    range: (0.000, 4.000)
}


In [7]:
# Converting a floating parameter into a fixed parameter.
param_b.make_fixed(initial=42.3)
print(param_b)

Parameter: b = 42.300 [fixed]


Comparing parameters
---

Two parameters can be compared to each other. They are equal if they represent the same parameter, i.e. the same name, initial value, and value range if they are floating parameters.

In [8]:
param_c = Parameter(name='b', initial=42.3)
print(param_c == param_b)
print(param_c is param_b)
print(param_c == param_a)

True
False
False


Creating a linear grid from a floating parameter
---

In [9]:
param_grid_a = param_a.as_linear_grid(delta=0.1)
print(param_grid_a)

a = [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5
 3.6 3.7 3.8 3.9 4. ], decimals = 1


Parameter Sets
---

In [10]:
from skyllh.core.parameters import ParameterSet

In [11]:
help(ParameterSet)

Help on class ParameterSet in module skyllh.core.parameters:

class ParameterSet(builtins.object)
 |  ParameterSet(params=None)
 |  
 |  This class holds a set of Parameter instances.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, params=None)
 |      Constructs a new ParameterSet instance.
 |      
 |      Parameters
 |      ----------
 |      params : instance of Parameter | sequence of Parameter instances | None
 |          The initial sequence of Parameter instances of this ParameterSet
 |          instance.
 |  
 |  __iter__(self)
 |      Returns an iterator over the Parameter instances of this ParameterSet
 |      instance.
 |  
 |  __len__(self)
 |      The number of parameters this ParameterSet has.
 |  
 |  __str__(self)
 |      Creates and returns a pretty string representation of this
 |      ParameterSet instance.
 |  
 |  add_param(self, param, atfront=False)
 |      Adds the given Parameter instance to this set of parameters.
 |      
 |      Parameters
 |      --

As example we are creating a set of two Parameters, $n_{\mathrm{s}}$, and $\gamma$.

In [12]:
param_ns = Parameter('ns', 100, valmin=0, valmax=1000)
param_gamma = Parameter('gamma', 2, valmin=1, valmax=4.)
print(param_ns)
print(param_gamma)

Parameter: ns = 100.000 [floating] {
    initial: 100.000
    range: (0.000, 1000.000)
}
Parameter: gamma = 2.000 [floating] {
    initial: 2.000
    range: (1.000, 4.000)
}


In [13]:
paramset = ParameterSet((param_ns, param_gamma))
print(paramset)

ParameterSet: 2 parameters (2 floating, 0 fixed) {
    Parameter: ns = 100.000 [floating] {
        initial: 100.000
        range: (0.000, 1000.000)
    }
    Parameter: gamma = 2.000 [floating] {
        initial: 2.000
        range: (1.000, 4.000)
    }
}


Parameter to Model mapping
--

As example we consider the following mapping of parameters to models and sources:

In [14]:
from skyllh.core.model import Model, SourceModel
from skyllh.core.parameters import ParameterModelMapper

We define the models and sources to which we want to map parameters:

In [15]:
model1 = Model('model1')
source1 = SourceModel('source1')
source2 = SourceModel('source2')

Now we define the parameters, which we want to map to the models and sources:

In [16]:
param_ns = Parameter('ns', 10, valmin=0, valmax=1000)
param_gamma1 = Parameter('gamma1', 2.5, valmin=1, valmax=4)
param_gamma2 = Parameter('gamma2', 3.3, valmin=1, valmax=4)
param_p = Parameter('p', 42)

After creating the models and parameters we can create the `ParameterModelMapper` for the set of models.

In [17]:
pmmapper = ParameterModelMapper(models=(model1,source1,source2))
print(pmmapper)

ParameterModelMapper: 0 global parameters, 3 models (2 sources)


Finally we can define the parameter mapping to the models for each parameter:

In [18]:
(pmmapper.def_param(param_ns, models=(source1,source2), model_param_names='ns')
         .def_param(param_gamma1, models=(source1,), model_param_names='gamma')
         .def_param(param_gamma2, models=(source2,), model_param_names='gamma')
         .def_param(param_p, models=(model1,)))
print(pmmapper)

ParameterModelMapper: 4 global parameters, 3 models (2 sources)
    Parameters:        
        ns [floating (0.000 <= 10.000 <= 1000.000)]
            in models:
            - source1: ns
            - source2: ns
                    
        gamma1 [floating (1.000 <= 2.500 <= 4.000)]
            in models:
            - source1: gamma
                    
        gamma2 [floating (1.000 <= 3.300 <= 4.000)]
            in models:
            - source2: gamma
                    
        p [fixed (42.000)]
            in models:
            - model1: p
            
